In [4]:
import numpy as np
import pandas as pd

Read in the data

In [5]:
data = pd.read_csv('1976_2018_house.csv', encoding= 'unicode_escape')

Choose the years after 2012

In [6]:
data = data.loc[data.year >=2014,:]

Fill the party NAs with 'I' for Independents

In [7]:
data['party']=data.party.fillna('I')

Iterate through the party column to choose the first letter for party

In [8]:
data['party']= [x[0].upper() for x in data['party']]

Choose the biggest political parties in the U.S.

In [9]:
data = data.loc[((data['party'] =='D') | (data['party'] == 'R') | (data['party'] == 'L') | (data['party'] == 'I') | (data['party'] == 'G') | (data['party'] == 'C')),:]

Drop columns we don't need

In [10]:
data = data.drop(columns = ["state_po", "state_ic", "office", "stage", "runoff", "special", "mode", "unofficial", "version", "writein", \
                           "state_fips", "state_cen"])

Rename the columns

In [11]:
data = data.rename(columns={"candidatevotes": "candidate_votes","totalvotes":"total_votes"})

Reset and set the index

In [12]:
data= data.reset_index()
data = data.drop(columns = ["index"])

Create the percent won column to get the winning party and candidate

In [13]:
data['pct_won']= round((data['candidate_votes']/data['total_votes'])*100,2)

Read in the state abbreviations dataset

In [14]:
state = pd.read_csv('state-abbrevs.csv')

Merge the two datasets

In [15]:
data = pd.merge(left=data, right=state, how='inner', on = 'state')

Create the state district column

In [16]:
data['state_dist'] = data['abbreviation'].astype(str) + '-'+ data['district'].astype(str)

Choose the candidates with the highest votes in each district for each year

In [17]:
data = data.groupby(['state_dist','year'])['candidate_votes', 'pct_won'].agg('max').reset_index()

C:\Users\devin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


Recreate the original dataframe

In [21]:
house_elections = pd.read_csv('1976_2018_house.csv', encoding= 'unicode_escape')
house_elections['party']=house_elections.party.fillna('I')
house_elections['party']= [x[0].upper() for x in house_elections['party']]
house_elections = house_elections.loc[((house_elections['party'] =='D') | (house_elections['party'] == 'R') | (house_elections['party'] == 'L') | (house_elections['party'] == 'I') | (house_elections['party'] == 'G') | (house_elections['party'] == 'C')),:]
house_elections = house_elections.drop(columns = ["state_po", "state_ic", "office", "stage", "runoff", "special", "mode", "unofficial", "version", "writein", \
                           "state_fips", "state_cen"])
house_elections = house_elections.rename(columns={"candidatevotes": "candidate_votes","totalvotes":"total_votes"})
house_elections= house_elections.reset_index()
house_elections = house_elections.drop(columns = ["index"])
house_elections['pct_won']= round((house_elections['candidate_votes']/house_elections['total_votes'])*100,2)
house_elections = pd.merge(left=house_elections, right=state, how='inner', on = 'state')
house_elections['state_dist'] = house_elections['abbreviation'].astype(str) + '-'+ house_elections['district'].astype(str)


Save the original dataframe into data2

In [22]:
data2 = house_elections

Merge the groupby dataframe and the original dataframe

In [23]:
data3 = pd.merge(left=data2, right=data, how='inner', left_on=['candidate_votes','state_dist', 'year'], right_on=['candidate_votes','state_dist', 'year'])

Drop the columns we don't need

In [24]:
data3 = data3.drop(columns = ['district', 'candidate_votes', 'candidate', 'pct_won_y', 'abbreviation', 'state', 'total_votes'])

Rename the pct_won column correctly

In [25]:
data3 = data3.rename(columns= {'pct_won_x': 'pct_won'})

In [29]:
data3

,year,party,pct_won,state_dist
0,2014,R,68.16,AL-1
1,2014,R,67.34,AL-2
2,2014,R,66.12,AL-3
3,2014,R,98.57,AL-4
4,2014,R,74.42,AL-5
...,...,...,...,...
1300,2018,R,60.11,WI-7
1301,2018,R,63.69,WI-8
1302,2014,R,66.05,WY-0
1303,2016,R,60.35,WY-0


Save the dataset

In [26]:
data3.to_csv("house_elections_final.csv", index=False)